In [3]:
#Update your token
STUDENT_TOKEN = 'DANIEL KUMLIN'

# Server Code

In [23]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print(f'Current game.player {game.player}')
        print('\033[92mYou won\033[0m' if game.player == player else '\033[91mYou lost\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

# Stratego Game

In [24]:
from functools import reduce
from copy import copy, deepcopy
import json


class Stratego(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return self.state['possible_actions']

  def card_str(self, card):
    return '+'.join(map(str, card))

  def print_game(self):
    print('Board: ' + "\n".join("".join(row) for row in self.state['board']))
    pprint.pprint(self.state)
    if self.state['past_actions']:
      print('Last action: ' + str(self.state['past_actions'][-1]))
      print('Last two actions: ' + " ".join(str(action) for action in self.state["past_actions"][-2:]))

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'

  def get_board(self):
        # Since `state` is already a dictionary, just return it directly.
        return self.state

# Rules

- Each piece does not have a value like in traditional stratego
- There are set of rules of who can beat who

Similar to Stratego, but smaller board (8x8) and fewer pieces
Barrage rules (only 1 bomb and 1 miner)
1 Field Marshal (10), 1 General (9), 1 Miner (7), 2 Scouts, 1 Spy, 1 Bomb, 1 Flag
Goal is to capture flag, anyone can do that
We will be playing best of 3 round

#### Movement

- Flag and Bomb can't move
- General rules
  - Can't move to ocupied space
  - Can't move to lakes
  - Can't move in diagonal
- Miner, Spy, Genereal and Field Marshal
  - Move to adjacent squares
- Scouts 
  - can as move in a straight line (as long as not intrerupted), as many spaces as desired
  - Lakes interrupt the path

#### How pieces can attack

Attacks:
- Flag and Bomb can't attack
- Only Miner can beat Bomb
- Spy will beat Field Marshal and flag if attacking, but lose all other figths
- Scouts can move and attack in same turn, everyone else need to decide to either move or attack

#### What each piece means

- Empty space is "_"
- Lakes are "L"
- Pieces are:
- Field Marshal: "F"
- General "G"
- Miner "M"
- Scout "S"
- Spy "s"
- Bomb "B"
- Flag "f"

#### What each phase is included in game

Actions:
- Place phase
  - List of tuples. (x, y, piece)
  - "X" places on rows 0 and 1
  - "O" places on rows 6 and 7
  - Both place on cols 2 through 5
- Move/attack phase
  - Initial position, end position
  - If end position is another player piece, it is an attack
  - Attacks are resolved automatically by the server
- Server returns all past actions in order
  - (player, (start_x, start_y), (end_x, end_y), result)
  - Result is a string:
  - "Move" (which piece is secret)
  - "G --> M" (General attacked Miner)
  - "M |-- G" (Miner failed attack on General)
  - "G <-> G" (General attacked General)

# Stratego Bot

## Making types for elements

In [31]:
from enum import Enum
from copy import deepcopy

class PieceType(Enum):
    FLAG = 'f'
    BOMB = 'B'
    FIELD_MARSHAL = 'F'
    GENERAL = 'G'
    MINER = 'M'
    SCOUT = 'S'
    SPY = 's'
    LAKE = 'L'
    EMPTY = '_'
    UKNOWN = 'X'

## Defining the bot

### Simple heuristic version

In [55]:
from typing import List, Optional
import random

class StrategoBot:
    def __init__(self):
        self.PIECE_VALUES = {
            PieceType.FLAG: 0,
            PieceType.BOMB: -1,
            PieceType.FIELD_MARSHAL: 10,
            PieceType.GENERAL: 9,
            PieceType.MINER: 7,
            PieceType.SCOUT: 2,
            PieceType.SPY: 1,
            PieceType.LAKE: -2,
            PieceType.EMPTY: 0,
            PieceType.UKNOWN: -3
        }
        
        # Map string representations to PieceType
        self.PIECE_MAP = {
            'f': PieceType.FLAG,
            'B': PieceType.BOMB,
            'F': PieceType.FIELD_MARSHAL,
            'G': PieceType.GENERAL,
            'M': PieceType.MINER,
            'S': PieceType.SCOUT,
            's': PieceType.SPY,
            'L': PieceType.LAKE,
            '_': PieceType.EMPTY,
            'O': PieceType.UKNOWN,
            'X': PieceType.UKNOWN
        }

    def evaluate_move(self, game_state: dict, move: List[List[int]]) -> float:
        """Evaluate the value of a move"""
        board = game_state['board']
        from_pos, to_pos = move
        piece = board[from_pos[0]][from_pos[1]]
        target = board[to_pos[0]][to_pos[1]]
        
        score = 0.0
        
        # Base position evaluation
        if target == 'O':  # Potential attack
            if piece == 's':  # Spy attacking
                score += 8.0  # High value for potential Field Marshal kill
            elif piece == 'M':  # Miner
                score += 5.0  # Good value for potential bomb
            else:
                score += 3.0  # Standard attack value
        
        # Strategic positioning
        # Prefer moves towards opponent's side
        if piece in ['F', 'G', 'M']:
            score += (to_pos[0] - from_pos[0]) * 0.5  # Reward forward movement
        
        # Scout special handling
        if piece == 'S':
            # Reward longer moves for scouts
            distance = abs(to_pos[0] - from_pos[0]) + abs(to_pos[1] - from_pos[1])
            score += distance * 0.3
        
        # Protect flag
        if piece == 'F':  # Field Marshal
            flag_pos = self.find_piece(board, 'f')
            if flag_pos:
                dist_to_flag = self.manhattan_distance(to_pos, flag_pos)
                score += (8 - dist_to_flag) * 0.4  # Reward staying near flag
        
        return score

    def manhattan_distance(self, pos1: List[int], pos2: List[int]) -> int:
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def find_piece(self, board: List[List[str]], piece: str) -> Optional[List[int]]:
        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == piece:
                    return [i, j]
        return None

    def select_move(self, game_state: dict) -> List[List[int]]:
        """Select the best move from possible actions"""
        possible_actions = game_state['possible_actions']
        
        if not possible_actions:
            return None
        
        # Evaluate all possible moves
        move_scores = [(move, self.evaluate_move(game_state, move)) 
                      for move in possible_actions]
        
        # Add some randomization to avoid predictability
        best_moves = sorted(move_scores, key=lambda x: x[1], reverse=True)[:3]
        
        # Select randomly from top 3 moves
        selected_move = random.choice(best_moves)[0]
        return selected_move


### Minimax Alpha-beta pruning version

In [42]:
class StrategoBotMinimax:
    def __init__(self):
        # Keep existing initialization
        self.MAX_DEPTH = 3  # Adjust based on performance needs

        self.PIECE_VALUES = {
            PieceType.FLAG: 0,
            PieceType.BOMB: -1,
            PieceType.FIELD_MARSHAL: 10,
            PieceType.GENERAL: 9,
            PieceType.MINER: 7,
            PieceType.SCOUT: 2,
            PieceType.SPY: 1,
            PieceType.LAKE: -2,
            PieceType.EMPTY: 0,
            PieceType.UKNOWN: -3
        }
        
        # Map string representations to PieceType
        self.PIECE_MAP = {
            'f': PieceType.FLAG,
            'B': PieceType.BOMB,
            'F': PieceType.FIELD_MARSHAL,
            'G': PieceType.GENERAL,
            'M': PieceType.MINER,
            'S': PieceType.SCOUT,
            's': PieceType.SPY,
            'L': PieceType.LAKE,
            '_': PieceType.EMPTY,
            'O': PieceType.UKNOWN,
            'X': PieceType.UKNOWN
        }
    
    def find_piece(self, board: List[List[str]], piece: str) -> Optional[List[int]]:
        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == piece:
                    return [i, j]
        return None

    def manhattan_distance(self, pos1: List[int], pos2: List[int]) -> int:
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def minimax(self, game_state: dict, depth: int, alpha: float, beta: float, maximizing: bool) -> Tuple[Optional[List[List[int]]], float]:
        if depth == 0:
            return None, self.evaluate_position(game_state)
            
        possible_actions = game_state['possible_actions']
        if not possible_actions:
            return None, self.evaluate_position(game_state)

        best_move = None
        if maximizing:
            max_eval = float('-inf')
            for move in possible_actions:
                new_state = self.simulate_move(game_state, move)
                _, eval = self.minimax(new_state, depth - 1, alpha, beta, False)
                
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                    
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return best_move, max_eval
        else:
            min_eval = float('inf')
            for move in possible_actions:
                new_state = self.simulate_move(game_state, move)
                _, eval = self.minimax(new_state, depth - 1, alpha, beta, True)
                
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
                    
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return best_move, min_eval

    def simulate_move(self, game_state: dict, move: List[List[int]]) -> dict:
        """Simulates a move and returns new game state"""
        new_state = deepcopy(game_state)
        from_pos, to_pos = move
        piece = new_state['board'][from_pos[0]][from_pos[1]]
        new_state['board'][to_pos[0]][to_pos[1]] = piece
        new_state['board'][from_pos[0]][from_pos[1]] = '_'
        # Update possible_actions for new state
        new_state['possible_actions'] = game_state['possible_actions']
        return new_state

    def evaluate_position(self, game_state: dict) -> float:
        """Evaluates entire board position"""
        score = 0.0
        board = game_state['board']
        
        # Material advantage
        for i in range(len(board)):
            for j in range(len(board[0])):
                piece = board[i][j]
                if piece in self.PIECE_MAP:
                    piece_type = self.PIECE_MAP[piece]
                    value = self.PIECE_VALUES[piece_type]
                    # Add value for our pieces, subtract for opponent's
                    if piece != 'O':
                        score += value
                    else:
                        score -= value * 0.8  # Unknown opponent pieces weighted less

        # Add positional evaluation
        score += self.evaluate_position_tactical(game_state)
        return score

    def evaluate_position_tactical(self, game_state: dict) -> float:
        """Evaluates tactical aspects of position"""
        score = 0.0
        board = game_state['board']
        
        # Flag protection
        flag_pos = self.find_piece(board, 'f')
        if flag_pos:
            for piece in ['F', 'B', 'G']:
                guard_pos = self.find_piece(board, piece)
                if guard_pos:
                    dist = self.manhattan_distance(flag_pos, guard_pos)
                    score += (4 - dist) * 0.5

        # Control of center
        center_squares = [(3,3), (3,4), (4,3), (4,4)]
        for i, j in center_squares:
            if board[i][j] not in ['_', 'O', 'L']:
                score += 0.3

        return score

    def select_move(self, game_state: dict) -> List[List[int]]:
        best_move, _ = self.minimax(game_state, self.MAX_DEPTH, float('-inf'), float('inf'), True)
        return best_move

## Solver function to initiate game flow

In [56]:
def solve_game(game: Stratego) -> List[List[int]]:
    """Populate the board with random pieces"""
    if game.state['phase'] == "place":
        if game.player == 'X':
          positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
          positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        final_pos = []
        for pos in positions:
          final_pos.append((pos[0], pos[1], pieces.pop()))
        print(final_pos)
        return final_pos

    # If not in placing phase (always at the start) then populate the board
    bot = StrategoBot()
    game_state = game.get_board()
    return bot.select_move(game_state)

In [38]:

def solve_game_minimax(game: Stratego) -> List[List[int]]:
    """Populate the board with random pieces"""
    if game.state['phase'] == "place":
        if game.player == 'O':
          positions = [(y, x) for x in range(2, 6) for y in range(2)]
        else:
          positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
        pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
        random.shuffle(pieces)
        final_pos = []
        for pos in positions:
          final_pos.append((pos[0], pos[1], pieces.pop()))
        print(final_pos)
        return final_pos

    # If not in placing phase (always at the start) then populate the board
    bot = StrategoBotMinimax()
    game_state = game.get_board()
    return bot.select_move(game_state)

In [57]:
game_loop(solve_game, Stratego, 'stratego', multi_player=False, id=None)

Creating new game...
Joining game with id: 18489
Playing as X
Waiting for the other player to join...
Board: ________
________
________
__L__L__
__L__L__
________
________
________
{'board': [['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_']],
 'past_actions': [],
 'phase': 'place',
 'possible_actions': []}
Making next move...
[(0, 2, 'G'), (1, 2, 'F'), (0, 3, 'S'), (1, 3, 'M'), (0, 4, 's'), (1, 4, 'B'), (0, 5, 'f'), (1, 5, 'S')]
b'Valid move'
Board: __GSsf__
__FMBS__
________
__L__L__
__L__L__
________
__OOOO__
__OOOO__
{'board': [['_', '_', 'G', 'S', 's', 'f', '_', '_'],
           ['_', '_', 'F', 'M', 'B', 'S', '_', '_'],
     

In [64]:
game_loop(solve_game_minimax, Stratego, 'stratego', multi_player=True, id=None)

Creating new game...
Joining game with id: 18503
Playing as O
Waiting for the other player to join...
Board: __FMfS__
__GsSB__
________
__L__L__
__L__L__
________
________
________
{'board': [['_', '_', 'F', 'M', 'f', 'S', '_', '_'],
           ['_', '_', 'G', 's', 'S', 'B', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_']],
 'past_actions': [],
 'phase': 'place',
 'possible_actions': []}
Making next move...
[(7, 2, 'S'), (6, 2, 'F'), (7, 3, 's'), (6, 3, 'S'), (7, 4, 'M'), (6, 4, 'G'), (7, 5, 'B'), (6, 5, 'f')]
b"Not player's turn"
Board: __FMfS__
__GsSB__
________
__L__L__
__L__L__
________
________
________
{'board': [['_', '_', 'F', 'M', 'f', 'S', '_', '_'],
           ['_', '_', 'G', 's', 'S', 'B', '_', '_']

KeyboardInterrupt: 

# Random

In [12]:
def random_solver(game: Stratego):
  if game.state['phase'] == "place":
    if game.player == 'X':
      positions = [(y, x) for x in range(2, 6) for y in range(2)]
    else:
      positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
    pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
    random.shuffle(pieces)
    final_pos = []
    for pos in positions:
      final_pos.append((pos[0], pos[1], pieces.pop()))
    print(final_pos)
    return final_pos
  else:
    return random.choice(game.actions())

In [32]:
game_loop(random_solver, Stratego, 'stratego', multi_player=False, id=None)

Creating new game...
Joining game with id: 18409
Playing as X
Waiting for the other player to join...
Board: ________
________
________
__L__L__
__L__L__
________
________
________
{'board': [['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', 'L', '_', '_', 'L', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_'],
           ['_', '_', '_', '_', '_', '_', '_', '_']],
 'past_actions': [],
 'phase': 'place',
 'possible_actions': []}
Making next move...
[(0, 2, 'S'), (1, 2, 'G'), (0, 3, 's'), (1, 3, 'B'), (0, 4, 'F'), (1, 4, 'f'), (0, 5, 'M'), (1, 5, 'S')]
b'Valid move'
Board: __SsFM__
__GBfS__
________
__L__L__
__L__L__
________
__OOOO__
__OOOO__
{'board': [['_', '_', 'S', 's', 'F', 'M', '_', '_'],
           ['_', '_', 'G', 'B', 'f', 'S', '_', '_'],
     